### Demonstration of an evaluation based on the MCTSAgent

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'../../../')
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
from DeepCrazyhouse.src.domain.agent.neural_net_api import NeuralNetAPI
from DeepCrazyhouse.src.domain.agent.player.mcts_agent import MCTSAgent
from DeepCrazyhouse.src.domain.agent.player.raw_net_agent import RawNetAgent
from DeepCrazyhouse.src.domain.variants.game_state import GameState
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
from time import time
%matplotlib inline
plt.style.use('seaborn-whitegrid')
enable_color_logging()

In [ ]:
batch_size = 8 #32 #8
threads = 8 #16

In [ ]:
nets = []
for idx in range(1):
    nets.append(NeuralNetAPI(ctx='cpu', batch_size=batch_size))

In [ ]:
raw_agent = RawNetAgent(nets[0])

In [ ]:
mcts_agent = MCTSAgent(nets, threads=threads,
                       playouts_empty_pockets=4096*5, playouts_filled_pockets=4096*5,
                     cpuct=2.5, u_init_divisor=0.25, dirichlet_epsilon=.25, dirichlet_alpha=0.2, batch_size=batch_size, q_value_weight=0.7,
                     max_search_depth=40, temperature=.07, virtual_loss=3, verbose=True, temperature_moves=0, enhance_checks=True,
                     min_movetime=10000, use_pruning=False, opening_guard_moves=0)

In [ ]:
board = chess.variant.GiveawayBoard() #CrazyhouseBoard()

#board.push_uci('e2e4')
#board.push_uci('e7e6')

fen = 'rnbqkb1r/ppp1pppp/5n2/3P4/8/8/PPPP1PPP/RNBQKBNR/P w KQkq - 1 3'
fen = 'rnb2rk1/p3bppp/2p5/3p2P1/4n3/8/PPPPBPPP/RNB1K1NR/QPPq w KQ - 0 11'
#fen = 'r1b1kbnr/ppp1pppp/2n5/3q4/3P4/8/PPP1NPPP/RNBQKB1R/Pp b KQkq - 1 4'
#fen = 'r1b1k2r/ppp2ppp/2n5/3np3/3P4/2PBP3/PpPB1PPP/1Q2K1NR/QNrb b Kkq - 27 14'
#fen = 'r1bb4/ppp2pkp/5npb/4p3/4P3/2N5/PPP1BPPP/3RK2R/QNRqpnp w K - 3 16'
#fen = '3R1b2/1bP1kp2/3Npn1p/3p4/5p2/5N1b/PPP2P1P/5Q1K/RPRPRQPnpbp b - - 4 36'
#fen = 'r3k1nr/ppp2ppp/2n5/8/q7/Pp6/1PnRN1PP/1K3B1R/BBQBppppp b kq - 3 27'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1/Qrbbpbb b - - 0 27'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1/Qrbbpbb b - - 0 30'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/5P2/3P1p2/PP3RPP/RN4K1/QQprbbpbb b - - 1 30'

# Test positions
#fen = '3R1b2/1bP1kp2/3Npn1p/3p4/5p2/5N1b/PPP1QP1P/3R1RK1/QPpprnpbp b - - 0 29'
fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1[Qrbbpbb] b - - 3 30'
#fen = 'q6r/p2P1pkp/1p1b1n2/2p2B2/8/6n1/PPP2KPp/R4R2/PNNRPBPbqpp w - - 2 26'
#fen ='2kr1b2/1bp2p1p/p3pP1p/1p5Q/5B2/3B1p2/PPP2PrP/R4R1K/QNpnnnp w - - 0 18'
#fen = 'q6r/p2P1pkp/1p1b1n2/2p2B2/8/6n1/PPP2KPp/R4R2/PNNRPBPbqpp w - - 50 26'
#fen = 'r4r1k/ppp1q1bp/4Ppp1/1P6/2NN3P/2BPPb2/P2PNPpR/R3K3/PBQn b Q - 2 34'
#fen = 'r1bq1b1r/ppp1kPpp/4Pn2/n2Pp3/2B4n/3P4/PPP2PPP/RNBQK2R/ w KQ - 0 10'

#fen = 'r4rk1/ppp2pbp/1nnp4/6P1/4Pp2/1P1P1N1P/2P2PPK/R2Q1R2/QNbbpb w - - 0 23'
#fen = 'r4rk1/ppp2pPp/1nn3nP/4b3/5p1B/1P1P3P/2P2PPK/R2Q1R2/Qbpnpb w - - 2 37'
#fen = 'r4rk1/ppp2pPp/1nn2bnP/4b3/6Q1/1P1P1pBP/2P2PPK/R4R2/Qpnpb w - - 2 39'
#fen = 'B2q1r1k/p3pp2/6pb/3pB2p/3N4/B2NP1pn/P1PP2P1/R2Q3K/PPrnprp b - - 0 28'
#fen = 'r1bq3r/p1ppbkpp/2p2n2/8/4P3/8/PPP2PPP/RNBQK2R/NPPnbp w KQ - 2 8'
#fen = 'r1bqk2r/1pp1nppp/1p3p2/3p4/3P3n/1B6/PP2BPPP/RNBQ1KNR/Pp b kq - 0 14'
#fen = '3qk2r/1pp1nppp/1p3p2/3p4/3P3n/1B5P/RP2BPpP/1NBQ2KR/RBPPn b k - 2 18'
#fen = 'r2q2kb/ppp4p/1nn1P1p1/4p1N1/2Pb4/1P1P2NP/2PB1rP1/Q4RK1/RPppb w - - 0 27'
#fen = 'r1bqk1r1/ppppbppp/5n2/4nN2/2B1P3/2N5/PPP2PPP/R1BQK2R/Pp w KQq - 2 8'
#fen = 'r2q1rk1/pP2ppbp/2p3pb/2Q5/4Pp2/2NB1NpP/PPP3P1/R4RK1/NBpn b - - 0 19'
#fen = 'r1bqkb1r/ppp1pppp/2n2n2/3p4/3P4/2N2N2/PPP1PPPP/R1BQKB1R/ w KQkq - 6 4'
#fen = 'r1bq1b2/2p1pk2/p1p2npp/8/3P4/P4N2/1PP2PPP/R1B1K2R/RPNPbnqp w KQ - 26 0'
#fen = 'r5k1/ppp2Rp1/1bnp1qP1/2b1pP2/8/3P1N1p/PPP2PPP/R2Q1R1K/Nbbn w - - 42 0'
#fen = 'r2q3r/ppp1k1pp/2npN3/2b1p3/4P1n1/2NP4/PPP2PPP/R1B2RK1/BPqb w - - 0 11'
#fen = 'r2qk1r1/ppp2p2/2np3p/2bBp1p1/4P1bn/3P1NB1/PPP2PPP/R2Q1RK1/N w q - 0 12'
#fen = 'r2k3r/pppb2pp/5b2/6N1/4P2n/3Q4/PPP2PPP/R1B1K2R/PPNPqbnp w KQ - 0 14'
#fen = 'r1bqk2r/ppp1bppp/5n2/3p4/2BQP3/2N5/PPP2PPP/R1B1K2R/NPnp w KQkq - 0 8'
#fen = '3r1rk1/p1p1npp1/2pp3p/2b1p1NQ/4P3/3P1P2/PPP2P1P/R2Q1RK1/BBnbn w - - 2 18'
#fen = 'r2q1rk1/p1p1bpp1/2pp1n1p/2b1pN2/4P1nB/3P1N2/PPP2PPP/R2Q1RK1/B w - - 0 13'
#fen = 'r1bq1rk1/ppp2pp1/2np1n1p/2b1p1B1/2B1P3/2NP1N2/PPP2PPP/R2Q1RK1/ w - - 14 8'
#fen = 'r1bqk1r1/pppp1ppp/2n2n2/5N2/1bB1P3/2N4p/PPP2PPP/R1BQK1R1/P w Qq - 2 9'
#fen = 'r1bqk1r1/ppp2ppp/2n2n2/3p1N2/1bB1P3/2N4P/PPP2P1P/R1BQK1R1/PP w Qq - 0 10'
#fen = 'r1bqk2r/ppppbppp/2n2n2/8/2BNP3/8/PPP2PPP/RNBQK2R/Pp w KQkq - 1 6'
#fen = 'r1b1k1r1/pp1p1ppp/1q1B1nn1/1B3N2/4P3/2P1p3/P1P1QPpP/R3K1R1/BPPn w Qq - 34 18'
#fen = 'r1bqk3/ppp1bprp/3p1n2/5PP1/4P2n/2NQ4/PPP2PBP/R1B1K2R/Npp w KQq - 1 13'
#fen = 'r1bqk1nr/pppp1ppp/5b2/4nNP1/2B1P3/7p/PPP2PPP/RNBQK2R/ w KQkq - 1 8'
#fen = 'r1b1k1nr/ppp2ppp/5q2/3pnN2/4P3/7p/PPP2PPP/RNBQKB1R/Bp w KQkq - 1 10'
#fen = 'r3k2r/ppp2ppp/5n2/3pnb2/4P3/7p/PPP2PPP/RNBQKB1R/Qbnp w KQkq - 22 12'
#fen = 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR/ w KQkq - 2 2'
#fen = 'r1bq1rk1/ppp2ppp/2np4/2bBp3/4P1n1/3P1N2/PPP1QPPP/R1B2RK1/N w - - 2 10'
#fen = 'r1bq1rk1/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP1N2/PPP2PPP/R1BQ1RK1/ w - - 12 7'
#fen = 'r1bq1rk1/pppp1ppp/2n5/2bBp3/4P1n1/3P1N2/PPP2PPP/R1BQ1RK1/N w - - 2 9'
#fen = 'r1bqk1r1/ppppbppp/5n2/1B2nN2/4P3/2N4p/PPP2PPP/R1BQK2R/P w KQq - 16 9'
#fen = 'r1bqkb1r/ppp2ppp/3p4/6P1/4n3/2N5/PPP2P1P/R1BQK1Rb/NPNpp w Qkq - 2 13'

fen = 'r1b1kb1r/p1p1pppp/2N5/1B2N3/2pPn3/2PKB3/P1PP2p1/3q1rR1/QPPNP w - - 0 25' # mate threat
#fen = 'r1b1kb1r/p1p1pppp/2N5/4N3/2BPn3/2PKB3/P1PP2p1/3q1rR1/PQPPNP b - - 0 25'
#fen = 'r1b1kb1r/p1p1pppp/2N5/1B6/2NPn3/2PKB3/P1PP2p1/3q1rR1/PQPPNP b - - 1 25'
#fen = '3rkb1r/2pb1pp1/q2Pp3/3pP2p/3P4/2PPnP2/P1P1N1PP/R2R2K1/NNQbb b k - 0 21' # mate threat

# previous stalled connection fen
#fen = 'r1bk3r/ppppbpQp/4p3/8/4n3/4P2N/PPPP2PP/R1Bq1BKR/PNNp b - - 2 13'
#fen = 'r1bqkbnr/ppp2ppp/3p4/8/3QP3/2N4p/PPP2PPP/R1B1KB1R/PNn w KQkq - 1 7'
#fen = 'r1bq1rk1/ppp2pp1/2np1n1p/2b1p1B1/2B1P3/2NP1N2/PPP2PPP/R2Q1RK1/ w - - 14 8'
#fen = 'rnb1kb1r/ppp1pppp/5n2/q7/8/2N2N2/PPPP1PPP/R1BQKB1R/Pp w KQkq - 0 5'
#fen = 'rnb2b1r/ppp1pkpp/5n2/2q5/3N2p1/2N5/PPPP1PPP/R1BQK2R/PPb w KQ - 0 8'
#fen = 'r1b1kr2/ppp3pp/3bPq2/1N1Ppn1n/2B4n/3PPP2/PPP2QPP/R1B2RK1/ b - - 9 15'
#fen = 'r1bk4/p1p1qP1p/2pn1p2/5pB1/2B1P3/8/PPPQ1PnP/R1B1rK2/RNPPPn w - - 0 23'
#fen = 'r1b1k2r/p1p1qppp/2p4P/5p2/2B1n3/5p2/PPP1QPPP/R1B1K2R[BNPnn] w KQkq - 0 12'
#fen = 'r1bbk2r/pp4pp/3Q4/1N2N3/7n/8/PPP2PpP/R1B3K1[PPPqrbnppp] w - - 1 20'
#fen = 'r2qkb1r/ppp3pp/4Pn2/4p3/2Bn4/8/PPPP1PPP/RNBQK2R/BPnp b KQkq - 0 8'
#fen = 'r2q1b1r/ppp1kBpp/4Pn2/3pp3/2Bn4/4P3/PPPP1PPP/RNBQK2R/n b KQ - 0 10'
#fen = 'r2qkb1r/ppp3pp/4Pn2/4p3/2Bn1n2/8/PPPP1PPP/RNBQK2R/BPp w KQkq - 0 9'
#fen = 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R/ b KQkq - 5 3'
#fen = 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R/ w KQkq - 6 4'
#fen = 'r2q1br1/ppp1k1pp/3p4/4p2Q/2BB1N1n/8/PPPP1P1P/RNB2K1R/ppnp b - - 0 17'
#fen = 'r2q1br1/ppp3pp/3pk3/4p3/2BB1N1n/8/PPPP1P1P/RNBQ1K1R/ppnp b - - 0 16'
#fen = 'r2qkb1r/ppp3pp/4Pn2/4p3/2Bn4/8/PPPP1PPP/RNBQK2R/BPnp b KQkq - 0 8'
#fen = 'r1b1k1q1/pp1n1prp/3Ppb1Q/bR4NB/8/8/PpP1pPPP/1R1q~N1K1/PPn w q - 0 29'
#fen = 'r1bbk3/pppn1prp/4p3/1p4B1/3PN3/5N2/PpPR1PPP/5RK1/QPpqnb w q - 0 18'
#fen = '2r5/p1p1qpPp/1k2p2P/1q1p3N/2RP4/2n4P/PpP2P1P/4R2K/NBBBBPNr w - - 0 37'
#fen = 'r4rk1/pp3pbp/2p2p1p/3q1N2/5P2/b1P3P1/P5P1/R2Q2K1/PPRPBBNNnp w - - 0 36'
#fen = 'rnbqkbnr/ppp1pppp/8/3p4/3PP3/8/PPP2PPP/RNBQKBNR/ b KQkq - 3 2'
#fen = 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R/ b KQkq - 5 3'
#fen = '2rq1rk1/ppp1bppp/2n1pn2/1N1p4/3P1B2/3PP1B1/PP2NPPP/R2Q1RK1[b] b - - 2 10'
#fen = 'rnbqkb1r/ppp2ppp/3ppn2/8/3PP1P1/2N5/PPP2P1P/R1BQKBNR/ b KQkq - 3 4'
#fen = 'r2qkb1r/pppn1ppp/4pn2/3p4/3N4/2N1P2B/PPPPQPPP/R1B1K2R/b b KQkq - 0 7'
#fen = 'r1bq1r1k/ppp2p1p/3p1p2/4p3/2BbP1nB/2PP4/PP1Q1PKP/R4R2[NNNp] b - - 0 16'
#fen = 'r2q3k/ppp2pRp/3p1p2/4p2N/4P2B/2PP1p1K/PP1b3P/R7[RBNNNPqb] b - - 0 29'
fen = '5r2/pppQ1p1k/3p1pPp/4p1n1/1b1BP3/2PP3p/PrNK2pN/r5B1/Qrbnp b - - 0 42'
fen = '2kr4/p1p2p2/5q2/3p1br1/1R1P1NpQ/8/P1PP1PpP/5RK1/BNNPPPbbnp w - - 1 26'
fen = 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R/ b KQkq - 5 3'
fen = 'r2q1kr1/pppb1pNp/2n1pp1P/1P6/1b1P3B/5P2/P1PQ1P1P/R4RK1/BNPn w - - 0 19'
fen = 'r4rk1/pp3pbp/2p2p1p/3q1N2/5P2/b1P3P1/P5P1/R2Q2K1/PPRPBBNNnp w - - 0 36'
fen = 'r4rk1/p1p2pNp/2p2p1p/6b1/3P2q1/3B4/P2Q1PPP/R4RK1/BNNPPbnpp w - - 1 24'
fen = 'r4rk1/p1p2p1p/2p2p1p/5Nb1/3P2q1/3B4/P2Q1PPP/R4RK1/BNNPPbnpp b - - 2 24'
fen = 'rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR/ w KQkq - 2 2'
fen = 'r1bq1k1r/p1ppbPpp/2p2n1P/4N3/4P3/8/PPP2PPP/RNBQK2R/nbp b KQ - 2 10'
#fen = 'r1bq1k1r/p1ppbPpp/2pb1n1P/4N3/4P3/8/PPP2PPP/RNBQK2R/np w KQ - 3 11'
#fen = 'r1bqk1nr/ppppbppp/2n5/8/2BNP3/8/PPP2PPP/RNBQK2R/Pp b KQkq - 9 5'
#fen = 'r1bq3r/ppppbkpp/2N2n2/8/4P3/8/PPP2PPP/RNBQK2R/NPPbp b KQ - 0 7'
#fen = 'r1bqk1nr/ppppbppp/2n5/8/2BNP3/8/PPP2PPP/RNBQK2R/Pp b KQkq - 0 5'
fen = 'r1bq2nr/ppp1bkpp/3p4/5P2/3QP3/8/PPP2PPP/RNB1K2R[NPbnp] b KQ - 0 8'
fen = 'r1bq1rk1/ppp1ppb1/2np2pp/4P3/2BP1BN1/2N2N1P/PPP2KP1/R2Q3R/p b - - 0 10'
fen = 'r3kb1r/pp1b1pp1/1b1Pp1p1/8/5B2/2N2pN1/PPP2P1P/R4RK1/NQPNPpq w kq - 0 22'
fen = 'r2q1b1r/ppp1p1pp/2n1kn2/1B2n3/8/2p2Q1P/PPPP1PP1/R1B1K2R/Bppn w KQ - 0 11'
fen = 'r2q1rk1/ppp2pp1/3p3B/3Pp2b/8/2PP1NpP/P5P1/R2Q1RK1[BNNPbnp] b - - 0 15'
fen = 'r2q1rk1/ppp2pp1/3p3B/3Pp2b/8/2PP1NpP/P5P1/R2Q1RK1/BNNPbnp b - - 0 15'
#fen = 'r2q1rk1/ppp2pp1/3p3B/3Pp2b/8/2PP1NpP/P4nP1/R2Q1RK1/NPNBbp w - - 0 16'
fen = 'rnbqk1r1/p4ppp/2p4P/2b1P3/2Bpn3/5N2/PPP2PPP/RNBQK2R[pp] w KQq - 0 10'
fen = 'r2qkb1r/ppp3pp/2n3bN/3pPR2/3P4/2QPP3/P5pP/R2K4[BBNNPPp] b - - 0 25'
fen = 'r2qkb1r/ppp3pp/2n3bN/3pPR2/3P4/2QPP3/P6P/R2KB1q~1/BNNPPp b - - 0 26'
#fen = 'r1bqkb1r/ppp1pppp/2n5/1B1n4/8/2N2N2/PPPP1PPP/R1BQK2R/Pp b KQkq - 9 5'
#fen = 'r1bqkb1r/ppp1pppp/2n5/3n4/8/2N2N2/PPPP1PPP/R1BQKB1R/Pp w KQkq - 0 5'
#fen = 'r1b1kb1r/ppp1pppp/2B5/3q4/5n2/2P2N2/P1PP1P1P/R1BQK1R1/NNPPp b Qkq - 0 10'
#fen = 'r1b1kb1r/p1p1pppp/2p5/3q4/5n2/2P2N2/P1PP1P1P/R1BQK1R1/NNPPbp w Qkq - 0 11'
#fen = 'r1bqkb1r/ppp1pppp/2n5/1B6/8/2P2N2/P1PP1PRP/R1BQK3/NPPnp b Qkq - 0 8'
#fen = 'r1b1kb1r/p1p1pppp/2p4q/8/1P1P1Nb1/2P2N2/P1P2PNP/R1BQK1R1/Pnp w Qkq - 0 14'
#fen = 'r1b1kb1r/p1p1pppp/2p5/7q/1P1P1N2/2P1NN2/P1P2PpP/R1BQK1R1/bnp b Qkq - 0 12'
#fen = 'r1bn1rk1/1ppp1ppp/p7/4p3/2B1P1P1/3PPN2/PPP3P1/RN1Q1RK1/BNQb b - - 0 10'
#fen = 'r4k2/ppp3pr/2p5/q4n2/2bp1Bn1/2N2B1p/PPPB1PpP/R3R1K1/Pnqpp w - - 0 23'
fen = 'r2qk2r/ppp3pp/2n2NbN/3pPR2/1b1P4/2QPP3/P6P/R2KB1q~1/BNPPp b - - 0 27'
fen = 'r2qk2r/ppp4p/2n2pbN/3pPR2/1b1P4/2QPP3/P6P/R2KB1q~1/BNPPnp w - - 0 28'
fen = 'r1bq4/2p2rk1/p2p2p1/1p2p1N1/2bnP2P/2N1K3/PPPQ1P2/3R3R/BBNPppp w - - 0 21'
fen = 'r1bq1rk1/pppp1ppp/2n1pn2/8/1b1PP3/2NBB3/PPP2PPP/R2QK1NR/ w KQ - 0 6'
fen = 'r2qkb1r/2p2ppp/p1p1p3/4P3/3Pp3/4BnN1/PPP1N1NP/5R1K/PRBBPq b kq - 0 19'
#fen = 'r2qkb1r/2p2ppp/p1p1p3/4P3/3Pp3/4B1N1/PPP1N1Nn/5R1K/PRBBPpq w kq - 0 20'
fen = 'r2qkb1r/1Bp2ppp/p1p1p3/4P3/3Pp3/4BrNK/PPP1N1N1/5N2/RQPBPp b kq - 0 25'
fen = 'r4k1r/p1p3p1/2p1Bqp1/3p4/3N3P/2N1P1b1/PPPPQPP1/R1B1K2R[Pbnnp] b KQ - 0 22'
fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P1p1/PPPPK1P1/R1B4R/QBPqbnnp b - - 0 25'
#fen = 'r4k1r/p1p3p1/2p1Bqp1/3p4/3N3P/2N1P3/PPPPQbP1/R1B1K2R/Ppbnnp w KQ - 0 23'
#fen = 'r4k1r/p1p3p1/2p1Bqp1/3p4/3N3P/2N1P3/PPPP1QP1/R1B1K2R/BPpbnnp b KQ - 0 23'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P3/PPPP1qP1/R1B1K2R/BPqpbnnp w KQ - 0 24'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P3/PPPP1KP1/R1B4R/QBPqpbnnp b - - 0 24'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P1K1/PPPP2P1/R1B4R/PQBPqbnnp b - - 0 25'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p3n/3N3P/2N1P1K1/PPPP2P1/R1B4R/PQBPqbnp w - - 0 26'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P1p1/PPPPK1P1/R1B3nR/QBPqbnp w - - 0 26'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p3b/3N3P/2N1P1p1/PPPPK1P1/R1B4R/QBPqnnp w - - 0 26'
#fen = 'r4k1r/p1p3p1/2p1Bqp1/3p4/3N3P/2N1P3/PPPPQbP1/R1BK3R/Ppbnnp b - - 0 23'
#fen = 'r4k1r/p1p3p1/2p1Bqp1/3p3b/3N2PP/2N1P1n1/PPPPQbP1/R1BK3R/pnp w - - 0 25'
#fen = '3rkb1r/ppp2ppp/2n5/3NPb2/4n3/8/PPPQPPPP/R3KB1R[QBPPn] w KQk - 0 12'
#fen = '3rkb1r/2pb1pp1/q2Pp3/3pP2p/3P4/2PPnP2/P1P1N1PP/R2R2K1/NNQbb b k - 0 21'
#fen = 'r1b1k2r/pppbnppp/4p3/3pP3/3Pb3/5Q2/PPP2P1P/RN4KR/BPNQn w kq - 0 17'
#fen = 'r1b1k2r/pppbn1pp/4p3/3pP1N1/3Pb3/8/PPP2P1P/RN4KR/PBPQqn w - - 0 19'
#fen = 'r3k2r/pppbqppp/2nbpn2/1N1p4/3P4/P2BPN1P/1PP2PP1/R1BQK2R b KQkq - 4 8'
#fen = 'r1bn1r2/ppp1bkp1/5n1p/8/2B1p2n/1PN1PpQP/PBP2PP1/R4RK1/PPq b - - 1 15'
#fen = 'r4k1r/p1p3p1/2p1B1p1/3p4/3N3P/2N1P1p1/PPPPK1P1/R1B4R/QBPqbnnp b - - 0 25'
#fen = '5b1k/prp1p1p1/2n3n1/8/8/2P1BN2/P1PP1P1P/R1BQK3/QPRrnpbpppp w Q - 0 27'
#fen = 'r1b1k2r/pppbnppp/4p3/3pP3/3P4/5Q2/PPP2P1P/RN4KR/BPNQbn b kq - 0 16'
#fen = 'r1b1k2r/pppbnppp/4p3/3pP3/3Pb3/5Q2/PPP2P1P/RN4KR/BPNQn w kq - 0 17'
#fen = '3rkb1r/2pb1pp1/q2Pp3/3pP2p/3P4/2PPnP2/P1P1N1PP/R2R2K1/NNQbb b k - 0 21'
#fen = '5b1k/prp1p1p1/2n3n1/8/8/2P1BN2/P1PP1P1P/R1BQK3/QPRrnpbpppp w Q - 0 27'
fen = 'r1b1k2r/ppp1qppp/2n1p3/2BpPn2/3P4/3B1N2/PPP2PPP/RN1QK2R/b b KQkq - 0 8'
#fen = 'r1bq2r1/ppp1kpPp/2n1pN2/6pN/3Pp3/5N2/P1P2PPP/R1B2RK1/qbbp w - - 0 17'
fen = 'r2q1rk1/p1p1bppp/2N2p2/3p4/8/2N5/PPPP1PPP/R1BQK1NR/BPnbp b KQ - 0 10'
fen = 'r2q1rk1/p1p1bppp/2N2p2/3p4/7n/2N5/PPPP1PPP/R1BQK1NR/BPbp w KQ - 0 11'
fen = 'r2N1rk1/p1p1bppp/5p2/3p4/7n/2N5/PPPP1P1P/R1BQ1KNR/QBPpbp w - - 0 13'
fen = '2r4k/pppb2p1/1bnp2Pp/4p3/1P2P2N/2PP4/q2N1PPP/4RR1K[RNqbbp] w - - 0 18'
fen = '2r4k/pppb2p1/1bnp2Pp/4p3/1P2P2N/2PP4/q2N1PPP/4RR1K[RNqbbp] w - - 0 18'
#fen = '2r3k1/pppb2pR/1bnp2Pp/4p3/1P2P2N/2PP4/q2N1PPP/4RR1K/Nqbbp w - - 0 19'
#fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR/ w KQkq - 0 1'
#fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNB1KBNR w KQkq - 0 1'
#fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKB1R/ w KQkq - 0 1'
#fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQK1NR w KQkq - 0 1'
fen = 'r2qkb1r/pppb1ppp/2n1p3/1B6/3P4/4NN2/PPP2PPP/R1BQK2R[Pnp] w KQkq - 3 9'
fen = 'rknbrqbn/pppppppp/8/8/8/8/PPPPPPPP/RKNBRQBN/ w - - 56 29'
fen = 'bb1q1r2/p3ppkp/8/1n3P2/4p1NP/1P4P1/P2PPP2/4NRKR[RNqbbppp] w - - 0 55'
fen = 'bb1qnrkr/pp2pppp/8/5P1N/4p1NP/1P4P1/P2PPP2/2Q1NRKR/Pbb w - - 0 47' # @h6 winning
fen = '3k2r1/pBpr1p1p/Pp3p1B/3p4/2PPn2B/5NPp/q4PpP/1R1QR1K1/NNbp w - - 1 23' # N@e6 winning
#fen = 'rnbqkbnr/ppp2ppp/4p3/3p4/4P3/2N5/PPPP1PPP/R1BQKBNR/ w KQkq - 4 3'
#fen = 'r1b1k1r1/ppp1b1pp/2n5/3PPp1N/2b1pP1n/2P1B3/PPQ2PPP/R2R2K1/Qn b q - 0 22'
#fen = 'r1b1kb1r/1pp2pPp/p1n2q2/8/8/2PB1p2/PP3PPP/R1BQK2R/PNPpnn w KQkq - 22 12'
#fen = 'r2q1r1k/1p3pp1/1p1p1b1p/p2P1Bn1/P3bP1Q/1Bp3P1/1PP5/R3R1K1/NPNpn w - - 0 29'
#fen = 'r2qkb1r/2p2ppp/p1p1p3/4P3/3PpN2/2N5/PPP1N2P/R1B2QK1/BPNrpb b kq - 0 13'
#fen = 'r2qkb1r/2p2ppp/p1p1p3/4P3/3PNN2/8/PPP1Q2P/R1B3K1/PPBPNnrb b kq - 0 15'
#fen = 'r2qkb1r/2p2ppp/p1p1p3/4P3/3PpN2/4BnN1/PPP1N2P/5R1K/RBBPqp b kq - 0 18'
#fen = 'r2qkb1r/2p3pp/p1p1p3/4P2B/3P1R2/4BpN1/PPP1N2K/8/NPPNPrbqp b - - 0 25'
#fen = 'r2q1knr/ppp4p/2npb3/2BpPp1b/3P1n2/2PB4/PP3PPP/R4RK1[Pqn] w - - 0 18'
#fen = 'r2q1r1k/1p3pp1/1p1p1b1p/p2P1Bn1/P3bP1Q/1Bp3P1/1PP5/R3R1K1/NPNpn b - - 0 29'
#fen = 'r7/1p2k3/p1prP2p/n3p2b/3nPbB1/3q2PP/P3ppPK/Q6R[rbnnppp] w - - 0 44'
#fen = 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4'
#fen = 'r1b1kbnr/pppp1ppp/2n5/1N2q3/5P2/4P3/PPP3PP/RNBQKB1R/pp b KQkq - 13 7'
#fen = ' r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPPBPPP/RNBQK2R/ b KQkq - 0 3'
#fen = 'rnbqkb1r/ppp1pppp/5n2/3P4/2P5/8/PP1P1PPP/RNBQKBNR/P b KQkq - 0 3'
#fen = 'r3r1k1/pppq1ppp/8/2Bpn1R1/3p4/1B1P2PP/PPPB2P1/RN1Q2K1/Bnnp b - - 2 18'
#fen = 'r3r1k1/pppq1ppp/8/2Bpn1R1/3p4/1B1P2PP/PPPB2PK/RN1Q4/PBnn b - - 4 19'
fen = 'r1bq2k1/ppp1nrpp/3pb3/6N1/P2NPp2/3P3P/1PP2PP1/RN1Q1RK1/Pbb b - - 0 14'
fen = 'r1bq1rk1/ppp1n1pp/4B3/4p1N1/P3Pp2/1P1P3P/RP3PP1/1N1Q1RK1/Pbbn b - - 0 22'
fen = 'r4Nk1/ppp1nbpp/3q4/4p3/P1B1P3/1P1P1P1P/RP3P2/1N1Q1RK1/RPPbbn b - - 0 26'
fen = '3RRb2/ppp1nkpp/3q4/4p3/P3Pn2/1P1P1P1P/RP3P2/1N1Q1RK1/PPbbbn b - - 0 31'
#fen = 'r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 34'
fen = 'r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R[nbb] w KQkq - 1 34'
fen = 'r1bqk1nr/pppp1ppp/2n1p3/8/1b1PP3/2N5/PPP2PPP/R1BQKBNR/ w KQkq - 6 4'

#board.set_fen(fen)

#board = board.mirror()

state = GameState(board)
board

In [ ]:
len(state.get_legal_moves())

In [ ]:
from DeepCrazyhouse.src.domain.variants.input_representation import board_to_planes
from DeepCrazyhouse.src.domain.variants.constants import *

In [ ]:
a = board_to_planes(board, 0, False)

In [ ]:
a.max()

In [ ]:
a.shape

In [ ]:
a.sum()

In [ ]:
len(a.flatten())

In [ ]:
a.flatten()[:2000].sum()

In [ ]:
key = 0
for i, val in enumerate(a.flatten()[:3000]):
    key += i * val

In [ ]:
key

In [ ]:
board.fen()

In [ ]:
board.fullmove_number

In [ ]:
chess.COLOR_NAMES

In [ ]:
board.turn

In [ ]:
len(list(state.get_legal_moves()))

In [ ]:
import copy

In [ ]:
for mv in board.generate_legal_captures():
    print(mv)

In [ ]:
def get_check_moves(board, legal_moves):
    check_idcs = []
    for idx, move in enumerate(legal_moves):
        board_tmp = copy.deepcopy(board)
        board_tmp.push(move)
        if board_tmp.is_check():
            check_idcs.append(idx)
    return check_idcs

In [ ]:
mat = np.zeros(99)

In [ ]:
idcs = get_check_moves(state.board, state.get_legal_moves())

In [ ]:
def plot_moves_with_prob(moves, probs, only_top_x=None):
    
    # revert the ordering afterwards
    idx_order = np.argsort(probs)[::-1]
    
    if only_top_x is not None and only_top_x < len(idx_order):
        idx_order = idx_order[:only_top_x]
    
    #moves_ordered = moves[range(len(moves))] #idx_order[::-1]]
    probs_ordered = [] #probs[idx_order]
    
    moves_ordered = []
    for idx in idx_order:
        probs_ordered.append(probs[idx])
        moves_ordered.append(moves[idx])
        
    plt.barh(range(len(probs_ordered)), probs_ordered)
    plt.yticks(range(len(moves_ordered)), moves_ordered)


### Evalution using the raw network

In [ ]:
t_s = time()
pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = raw_agent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
len(state.get_legal_moves())

In [ ]:
cp

In [ ]:
pred_value

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

### Evalution using the MCTS-Agent

for i in range(10):
    value, selected_move, confidence, idx, centipawn, depth, nodes, time_elapsed_s, nps, pv = mcts_agent.perform_action(state)
    print("selected_move", selected_move.uci())

In [ ]:
l = [1,2,3,4,5]

In [ ]:
np.random.randint(0, 2)

In [ ]:
t_s = time()
pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = mcts_agent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
mcts_agent.root_node.legal_moves[mcts_agent.root_node.child_number_visits.argmax()]

In [ ]:
mcts_agent.root_node.legal_moves[mcts_agent.root_node.q_value.argmax()]

In [ ]:
from DeepCrazyhouse.src.domain.agent.player.alpha_beta_agent import AlphaBetaAgent

In [ ]:
# minimax(currentPosition, 3, -∞, +∞, true)

In [ ]:
ab_agent = AlphaBetaAgent(net=nets[0], depth=4, nb_candidate_moves=10, include_check_moves=False)

In [ ]:
state.board

In [ ]:
ab_agent.best_moves

In [ ]:
ab_agent.best_moves

In [ ]:
d = np.array([4,3,2,1])

In [ ]:
np.argsort(d)[::-1][:2]

In [ ]:
[2,3] + [3,4]

In [ ]:
l = [1,2,3]

In [ ]:
l[:-1]

In [ ]:
t_s = time()
pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = ab_agent.evaluate_board_state(
    state) #, depth=5)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
t_s = time()
pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = ab_agent.evaluate_board_state(
    state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
ab_agent.best_moves

In [ ]:
mcts_agent.root_node.n_sum

In [ ]:
board

In [ ]:
for i, child_node in enumerate(mcts_agent.root_node.child_nodes):
    if child_node.initial_value < 0:
        print(legal_moves[i], child_node.initial_value)

In [ ]:
mcts_agent.root_node.legal_moves[mcts_agent.root_node.child_number_visits.argmax()]

In [ ]:
mcts_agent.root_node.legal_moves[mcts_agent.root_node.q_value.argmax()]

In [ ]:
pb_c_base = 19652 / 10
pb_c_init = 3.6
import math
cpuct = math.log((3000 + pb_c_base + 1) / pb_c_base) + pb_c_init

In [ ]:
cpuct

In [ ]:
#c = (a * b ) + (0.3 * a + 0.7 * b)

In [ ]:
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [ ]:
mcts_agent.get_last_q_values(min_nb_visits=10)

In [ ]:
np.sort(mcts_agent.root_node.child_number_visits)[::-1][:100]

In [ ]:
plot_moves_with_prob(legal_moves, mcts_agent.root_node.child_number_visits, only_top_x=20)

In [ ]:
plot_moves_with_prob(legal_moves, mcts_agent.root_node.q_value, only_top_x=4)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:


%load_ext line_profiler
r = %lprun -r -f mcts_agent._run_single_playout mcts_agent._run_single_playout(mcts_agent.root_node)
r.print_stats()



In [ ]:
from DeepCrazyhouse.src.domain.crazyhouse.input_representation import board_to_planes


In [ ]:
%load_ext line_profiler
r = %lprun -r -f board_to_planes board_to_planes(board)
r.print_stats()

In [ ]:
%load_ext line_profiler
r = %lprun -r -f board_to_planes board_to_planes(board)
r.print_stats()

In [ ]:
import numpy as np

In [ ]:
pb_c_base = 19652
pb_c_init = 1.25

In [ ]:
vists = np.linspace(0, 1500)

In [ ]:
pb_c = np.log((vists + pb_c_base + 1) /
                  pb_c_base) + pb_c_init

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(vists, pb_c)

In [ ]:
plt.plot(vists, pb_c)